In [ ]:
##Links for requests
BTC = "https://api.github.com/repos/bitcoin/bitcoin"
SOL = "https://api.github.com/repos/solana-labs/solana"
ETH = "https://api.github.com/repos/ethereum"
DOGE = "https://api.github.com/repos/dogecoin/dogecoin"
MATIC = "https://api.github.com/repos/maticnetwork/bor"
STORJ = "https://api.github.com/repos/storj/storj"
AVAX = "https://api.github.com/repos/ava-labs"
SAND = "https://api.github.com/repos/thesandboxgame"
DYDX = "https://api.github.com/repos/dydxprotocol"
BNB = "https://api.github.com/repos/bnb-chain"


#Extension for different stats
yearly_commits = "/stats/commit_activity"
weekly_commits_aggregated = "/stats/code_frequency"
total_commits_users = "/stats/participation"
hourly_commits = "/stats/punch_card"
issues = "/issues"
issue_comments = "/issues/comments"


#Defining Data Frame for main data
data=pd.DataFrame()

#Defining DFs for Extensions Data
extension=pd.DataFrame()
##-- For Yearly Commits
yc_df=pd.DataFrame()
##-- For Weekly Commits Aggregated Data
wca_df=pd.DataFrame()
##-- For Total Commits Data
tcu_df=pd.DataFrame()
##-- For Hourly Commits Data
hc_df=pd.DataFrame()
##-- For Issues Data
issues_df=pd.DataFrame()
##-- For Issue Comments Data
issuecom_df=pd.DataFrame()

#Creating a loop to get the required data---

## Use this to fetch for all cryptos
#crypto = [BTC, SOL, ETH, DOGE, MATIC, STORJ, AVAX, SAND, DYDX, BNB]
## Use this for specific crypto
crypto = [BTC]

extensions = [yearly_commits, weekly_commits_aggregated , total_commits_users, hourly_commits, issues , issue_comments]
#extensions = [issue_comments]

for m in crypto :
    # Pulls data from GitHub main dataframe
    common = requests.get(m).json()
    d1 = json_normalize(common)  
    # Adds a column to the main dataframe with the Crypto name
    crypto_name=m.split('/')[-1]
    d1['Crypto']=crypto_name
    # Appends pulled data to the main dataframe
    data=data.append(d1)
    for n in extensions:
        # Pulls each extensions data table
        pull = requests.get(m + n).json()
        d2 = json_normalize(pull)  
        # Adds a column to each dataframe with the Crypto and extension name
        d2['Crypto_Name']=crypto_name
        d2['Extension']=n
        # Appends the pulled data to the corresponding pre-prepared dataframe
        if n==yearly_commits:
          yc_df=yc_df.append(d2)
        elif n==weekly_commits_aggregated:
          wca_df=wca_df.append(d2)
        elif n==total_commits_users:
          tcu_df=tcu_df.append(d2)
        elif n==hourly_commits:
          d2 = pd.DataFrame(pull, columns = ['Day','Hour', 'Number of Commits'])
          hc_df=hc_df.append(d2)
        elif n==issues:
          issues_df=issues_df.append(d2)
        elif n==issue_comments:
          issuecom_df=issuecom_df.append(d2)
        else:
          continue
                    
    print("Extension DF for crypto",m ,"done for:",n)
print("DFs for crypto done for:",m)        
     
        
        
#Preprocessing--
#1.For Yearly Commit Data Frame---
##Converting Days Column values in 'Yearly Commit' to separte columns
yc_df[['Sun','Mon','Tue','Wed','Thur','Fri','Sat']] = pd.DataFrame(yc_df.days.tolist(), index= yc_df.index)        

#2.For the main data frame -----
##Rearranging columns & Keeping only the features that are needed
data=data[['id', 'created_at','updated_at','pushed_at', 'size', 'stargazers_count','watchers_count', 'forks_count','open_issues_count',
           'watchers','network_count', 'subscribers_count','owner.id','organization.id','Crypto']]
#Renaming the columns
data = data.rename(columns = {'owner.id': 'owner_id','organization.id':'organization_id'}, inplace = False)

        
  
#3.For the Issues data frame -----      
issues_df=issues_df[['id','number','title','created_at','updated_at','closed_at','author_association','comments','body','user.login','user.id','Crypto_Name','Extension']]     
#Renaming the columns
issues_df = issues_df.rename(columns = {'user.login': 'user_login','user.id':'user_id'}, inplace = False)
     
    